In [1]:
import os
import re

import csv

import requests

from bs4 import BeautifulSoup

In [2]:
def get_text(soup):
    text = ""
    for paragraph in soup.find_all('p'):
        if not paragraph.has_attr('class') and not paragraph.findChildren('b'):
            text += paragraph.get_text() + "\n\n"
            
    return text

In [3]:
def get_npr_urls(soup):
    
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    urls = []
    for link in soup.find_all('a', href=True):
        url = link['href']
        if (url.startswith('https://www.npr.org/')):
            if re.search(pattern, url):
                urls.append(url)
        
    return urls

In [4]:
def get_title(soup):
    title = ""
    #find href with class == title, then get b
    for link in soup.find_all('a'):
        if link.has_attr('class'):
            if link['class'][0] == "title":
                title = link.findChildren('b')[0].get_text()
    
    return title

In [5]:
def get_details(url):
    pattern = r'20[\d]{2}/[\d]{1,2}/[\d]{1,2}/[\d]+/'
    
    matchObject = re.search(pattern, url, flags=0)
    match_split = matchObject[0].split("/")
    date = match_split[0] + "-" + match_split[1] + "-" + match_split[2]
    article_id = match_split[3]
    
    return date, article_id

In [6]:
##############
## 

start_url = "https://www.npr.org/"
traversed_urls = set()

to_traverse = set()
to_traverse.add(start_url)

folder_out = 'data/articles/'
if not os.path.exists(folder_out):
    os.makedirs(folder_out)

article_count = 0

with open(folder_out + "article_log.csv", 'w') as csvfile:
    fieldnames = ["ID", "Date", "Title", "Link"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    while len(to_traverse) > 0 and article_count < 2000:
        if article_count % 50 == 0:
            print(article_count, ":", len(to_traverse))
        url = to_traverse.pop()
        traversed_urls.add(url)

        soup = BeautifulSoup(requests.get(url).text, 'html.parser')

        links = get_npr_urls(soup)

        urls = [x for x in get_npr_urls(soup) if x not in traversed_urls]
        to_traverse.update(urls)

        text = get_text(soup)

        if len(text) > 100:
            title = get_title(soup)
            date, article_id = get_details(url)

            writer.writerow({"Link": url, "Title": title, "Date": date, "ID": article_id})
            with open (folder_out + article_id + ".txt", 'w') as article_hdl:
                article_hdl.write(text)
            

        article_count += 1

0 : 1
50 : 69
100 : 162
150 : 253
200 : 303
250 : 387
300 : 444
350 : 445
400 : 471
450 : 466
500 : 538
550 : 604
600 : 622
650 : 646
700 : 744
750 : 791
800 : 872
850 : 927
900 : 932
950 : 950
1000 : 1038
1050 : 1167
1100 : 1211
1150 : 1261
1200 : 1283
1250 : 1331
1300 : 1381
1350 : 1437
1400 : 1469
1450 : 1446
1500 : 1549
1550 : 1584
1600 : 1584
1650 : 1639
1700 : 1677
1750 : 1728
1800 : 1781
1850 : 1839
1900 : 1957
1950 : 2003


In [7]:
import glob
len(glob.glob(folder_out + "*.txt"))

1938

In [8]:
with open (folder_out + "article_log.csv") as csv_hdl:
    count = 0
    reader = csv.reader(csv_hdl)
    for row in reader:
        count+=1
        
    print (count)

1988
